In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as pltgrap
from src.graph import NeighborsGraph

In [9]:
graph = NeighborsGraph.load('../data/datasets/dataset_graph.pkl')

In [11]:
graph.get_neighbor_distances('P09165')

array([1.37739063, 1.18412761, 1.40929805, 1.21011888])

In [ ]:
def figure(results_df:pd.DataFrame):
